In [ ]:
pip install apache_beam[interactive]

In [ ]:
import apache_beam as beam

In [3]:
p1 = beam.Pipeline()

# É preciso criar labels específicas para cada etapa e elas não podem ter o mesmo nome em momento algum

Tempo_Atrasos = (
p1
  | "(Tempo): Importar os dados" >> beam.io.ReadFromText("voos_sample.csv", skip_header_lines=1)
  | "(Tempo): Separar por vírgulas" >> beam.Map(lambda record: record.split(','))
  | "(Tempo): Pegar vôos de Los Angeles" >> beam.Filter(lambda record: int(record[8]) > 0)
  | "(Tempo): Criar o par" >> beam.Map(lambda record: (record[4], int(record[9])))
  | "(Tempo): Somar por key" >> beam.CombinePerKey(sum)
# Se mantiver o print, ele "perde" o status e não guarda o resultado para uso posterior
# | "Mostrar os resultados" >> beam.Map(print)
)


Qtd_Atrasos = (
p1
  | "(Qtd): Importar os dados" >> beam.io.ReadFromText("voos_sample.csv", skip_header_lines=1)
  | "(Qtd): Separar por vírgulas" >> beam.Map(lambda record: record.split(','))
  | "(Qtd): Pegar vôos de Los Angeles" >> beam.Filter(lambda record: int(record[8]) > 0)
  | "(Qtd): Criar o par" >> beam.Map(lambda record: (record[4], int(record[9])))
  | "(Qtd): contar por key" >> beam.combiners.Count.PerKey()
# | "Mostrar os resultados" >> beam.Map(print)
)

tabela_atrasos = (
    {'Qtd_Atrasos':Qtd_Atrasos, 'Tempo_Atrasos': Tempo_Atrasos}
    | "Group By" >> beam.CoGroupByKey()
    | beam.Map(print)
)

# para executar:
p1.run()

('LAX', {'Qtd_Atrasos': [4], 'Tempo_Atrasos': [1454]})
('HNL', {'Qtd_Atrasos': [1], 'Tempo_Atrasos': [510]})
('DFW', {'Qtd_Atrasos': [1], 'Tempo_Atrasos': [385]})
('OGG', {'Qtd_Atrasos': [1], 'Tempo_Atrasos': [497]})
('JFK', {'Qtd_Atrasos': [4], 'Tempo_Atrasos': [1102]})
